# Stommel Gyre on Unstructured Grid
This tutorial walks a simple example of using Parcels for particle advection on an unstructured grid. The purpose of this tutorial is to introduce you to the new way fields and fieldsets can be instantiated in Parcels using UXArray DataArrays and UXArray grids.

We focus on a simple example, using constant-in-time velocity and pressure fields for the classic barotropic Stommel Gyre. This example dataset is included in Parcels' new `parcels._datasets` module. This module provides example XArray and UXArray datasets that are compatible with Parcels and mimic the way many general circulation model outputs are represented in (U)XArray. 

## Loading the example dataset
Creating a particle simulation starts with defining a dataset that contains the fields that will be used to influence particle attributes, such as position, through kernels. In this example, we focus on advection. Because of this, the dataset we're using will provide velocity fields for our simulation.

Parcels now includes pre-canned example datasets to demonstrate the schema of XArray and UXArray datasets that are compatible with Parcels. For unstructured grid datasets, you can use the `parcels._datasets.unstructured.generic.datasets` dictionary to see which datasets are available for unstructured grids.

In [ ]:
from parcels._datasets.unstructured.generic import datasets as datasets_unstructured

In [ ]:
datasets_unstructured.keys()

In this example, we'll be using the stommel_gyre_delaunay example dataset. This dataset is created by generating a delaunay triangulation of a uniform grid of points in a square domain $x \in [0,60^\circ] \times [0,60^\circ]$. There is a single vertical layer that is 1000m thick. This layer is defined by the layer surfaces $z_f = 0$ and $z_f = 1000$.

In [ ]:
ds = datasets_unstructured["stommel_gyre_delaunay"]
ds

In [ ]:
print("lon", ds.uxgrid.face_lon.min().data[()], ds.uxgrid.face_lon.max().data[()])
print("lat", ds.uxgrid.face_lat.min().data[()], ds.uxgrid.face_lat.max().data[()])

In the dataset, we have the following dimensions

* `time: 1` - The number of time levels that the variables in this dataset are defined at. 
* `nz1: 1` - The number of vertical layers. The `nz1` dimension is associated with the `nz1` coordinate that defines the vertical position of the center of each vertical layer. The `nz1` coordinate consists of non-negative values that are assumed to increase with `nz1` dimension index.
* `n_face: 721` - The number of 2-d unstructured grid faces in the `UXArray.grid`
* `nz: 2` - The number of vertical layer interfaces. The `nz` dimension is associated with the `nz` coordinate that defines the vertical positions of the interfaces of each vertical layer. The `nz` coordinate consists of non-negative values that are assumed to increase with `nz` dimension index. Note that the number of layer interfaces is always the number of layers plus one.
* `n_node: 400` - The number of corner node vertices in the grid.

Whenever you are building a UXArray dataset for use in Parcels, its important to keep in mind that these dimensions and coordinates are assumed to exist for your dataset. Further, it is highly recommended that you use UXArray when possible to load unstructured general circulation model data when possible. This ensures that other characteristics, such as the counterclockwise ordering of vertices for each element, are defined properly for use in Parcels.

## Defining a Grid, Fields, and Vector Fields

A `UXArray.Dataset` consists of multiple `UXArray.UxDataArray`'s and a `UXArray.UxGrid`. Parcels views general circulation model data through the `Field` and `VectorField` classes. A `Field` is defined by its `name`, `data`, `grid`, and `interp_method`. A `VectorField` can be constructed by using 2 or 3 `Field`'s. The `Field.data` attribute can be either an `XArray.DataArray` or `UXArray.UxDataArray` object. The `Field.grid` attribute is of type `Parcels.XGrid` or `Parcels.UXGrid`. Last, the `interp_method` is a dynamic function that can be set at runtime to define the interpolation procedure for the `Field`. This gives you the flexibility to use one of the pre-defined interpolation methods included with Parcels v4, or to create your own interpolator. 

The first step to creating a `Field` (or `VectorField`) is to define the Grid. For an unstructured grid, we will create a `Parcels.UXGrid` object, which requires a `UxArray.grid` and the vertical layer interface positions. Setting the `mesh` to `"spherical"` is a legacy  feature from Parcels v3 that enables unit conversion from `m/s` to `deg/s`; this is needed in this case since the grid locations are defined in units of degrees.

In [ ]:
from parcels.uxgrid import UxGrid

grid = UxGrid(grid=ds.uxgrid, z=ds.coords["nz"], mesh="spherical")
# You can view the uxgrid object with the following command:
grid.uxgrid

With the `UxGrid` object defined, we can now define our `Field` objects, provided we can align a suitable interpolator what that `Field`. Aligning an interpolator requires you to be cognizant of the location that each `DataArray` is associated with. Since Parcels v4 provides flexibility to customize your interpolation scheme, care must be taken when pairing an interpolation scheme with a field. On unstructured grids, data is typically registered to "nodes", "faces", or "edges". For example, with FESOM2 data, `u` and `v` velocity components are face registered while the vertical velocity component `w` is node registered.

In Parcels, grid searching is conducted with respect to the faces. In other words, when a grid index `ei` is provided to an interpolation method, this refers the face index `fi` at vertical layer `zi` (when unraveled). Within the interpolation method, the `field.grid.uxgrid.face_node_connectivity` attribute can be used to obtain the node indices that surround the face. Using these connectivity tables is necessary for properly indexing node registered data.

For the example Stommel gyre dataset in this tutorial, the `u` and `v` velocity components are face registered (similar to FESOM). Parcels includes a nearest neighbor interpolator for face registered unstructured grid data through `Parcels.application_kernels.interpolation.UXPiecewiseConstantFace`. Below, we create the `Field`s `U` and `V` and associate them with the `UxGrid` we created previously and this interpolation method.

In [ ]:
from parcels.application_kernels.interpolation import UXPiecewiseConstantFace
from parcels.field import Field

U = Field(
    name="U",
    data=ds.U,
    grid=grid,
    interp_method=UXPiecewiseConstantFace,
)
V = Field(
    name="V",
    data=ds.V,
    grid=grid,
    interp_method=UXPiecewiseConstantFace,
)
P = Field(
    name="P",
    data=ds.p,
    grid=grid,
    interp_method=UXPiecewiseConstantFace,
)

Now that we've defined the `U` and `V` fields, we can define a `VectorField`. The `VectorField` is created in a similar manner, except that it is initialized with `Field` objects. You can optionally define an `interp_method` on the `VectorField`. When this is done, the `VectorField.interp_method` is used for interpolation; otherwise, evaluation of the `VectorField` is done component-wise using the `interp_method` associated with each component separately.

In [ ]:
from parcels.field import VectorField

UV = VectorField(name="UV", U=U, V=V)

## Defining the FieldSet
With all of the fields defined, that we want for this simulation, we can now create the `FieldSet`. As the name suggests, the `FieldSet` is the set of all `Field`s that will be used for a particle simulation. A `FieldSet` is initialized with a list of `Field` objects

In [ ]:
from parcels.fieldset import FieldSet

fieldset = FieldSet([UV, UV.U, UV.V, P])

## Setting your own custom interpolator
You may be wondering how to set your own custom interpolator. In Parcels v4, this is as simple as defining a function that matches a specific API. The API you need to match is defined in the `field.py` module in the `Field._interp_template` and `VectorField._interp_template`. Specifically,

```python
def _interp_template(
        self, # Field or VectorField
        ti: int, # Time index
        ei: int, # Flat grid index
        bcoords: np.ndarray, # Barycentric coordinates relative to the cell vertices
        tau: np.float32 | np.float64, # Time interpolation weight
        t: np.float32 | np.float64, # Current simulation time
        z: np.float32 | np.float64, # Current particle depth
        y: np.float32 | np.float64, # Current particle y-position
        x: np.float32 | np.float64, # Current particle x-position
    ) -> np.float32 | np.float64 # For `Field`, returns a float value.
```

So long as your function matches this API, you can define such a function and set the `Field.interp_method` to that function.




In [ ]:
import numpy as np


def my_custom_interpolator(
    self,
    ti: int,
    ei: int,
    bcoords: np.ndarray,
    tau: np.float32 | np.float64,
    t: np.float32 | np.float64,
    z: np.float32 | np.float64,
    y: np.float32 | np.float64,
    x: np.float32 | np.float64,
) -> np.float32 | np.float64:
    """Custom interpolation method for the P field.
    This method interpolates the value at a face by averaging the values of its neighboring faces.
    While this may be nonsense, it demonstrates how to create a custom interpolation method."""

    zi, fi = self.grid.unravel_index(ei)
    neighbors = self.grid.uxgrid.face_face_connectivity[fi]
    f_at_neighbors = self.data.values[ti, zi, neighbors]
    # Interpolate using the average of the neighboring face values
    if len(f_at_neighbors) > 0:
        return np.mean(f_at_neighbors)
    # If no neighbors, return the value at the face itself
    else:
        return self.data.values[ti, zi, fi]


# Assign the custom interpolator to the P field
P.interp_method = my_custom_interpolator

## Understanding the context inside an interpolator method
Providing the `Field` object as an input to an interpolator exposes you to a ton of useful information and methods for building complex interpolators. Particularly, the `Field.grid` attribute gives you access to connectivity tables and metric terms that you may find useful for constructing an interpolator. For context, the `Parcels.UXGrid` class is built on top of the `Parcels.BaseGrid` class (much likes it's structured grid `Parcels.XGrid` counterpart). The `Parcels.UXGrid` class combines a `UXArray.grid` object alongside the vertical layer interfaces, which provides sufficient information to define the API that the `BaseGrid` class demands. This includes

* `search` - A method for returning a flat grid index `ei` for a position `(x,y,z)`
* `ravel_index` - A method for converting a face index `fi` and a vertical layer index `zi` into a single flat grid index `ei`
* `unravel_index` - A method for converted a single flat grid index `ei` into a face index `fi` and a vertical layer index `zi`

The `ravel/unravel` methods are a necessity for most interpolators. For unstructured grids, the `Field.grid.uxgrid` attribute give you access to all of the attributes associated with a `UxArray.grid` object (See https://uxarray.readthedocs.io/en/latest/api.html#grid for more details.)

# Running the forward integration

In [ ]:
from parcels import AdvectionEE, AdvectionRK4

In [ ]:
from datetime import datetime, timedelta

from parcels import Particle, ParticleSet

In [ ]:
num_particles = 2

pset = ParticleSet(
    fieldset,
    lon=np.random.uniform(3.0, 57.0, size=(num_particles,)),
    lat=np.random.uniform(3.0, 57.0, size=(num_particles,)),
    depth=50.0 * np.ones(shape=(num_particles,)),
    time=0.0
    * np.ones(
        shape=(num_particles,)
    ),  # important otherwise initialization appears to take forever?
    pclass=Particle,
)
print(len(pset), "particles created")

In [ ]:
from tqdm import tqdm

from parcels import FieldOutOfBoundError

# for capturing positions
_lon = [pset.lon]
_lat = [pset.lat]

# output / sub-experiment time stepping
output_dt = timedelta(minutes=10)
endtime = output_dt

# run 40 x 6 hours and capture positions after each iteration
for num_output in tqdm(range(40)):
    # if one particle errors, let's top all of them
    try:
        pset.execute(
            endtime=endtime,
            dt=timedelta(seconds=60),
            pyfunc=AdvectionEE,
            verbose_progress=False,
        )
    except FieldOutOfBoundError:
        print("out of bounds, stopping (all particles)")
        break

    # on to the next sub experiment
    endtime += output_dt
    _lon.append(pset.lon)
    _lat.append(pset.lat)

# merge captured positions
lon = np.vstack(_lon)
lat = np.vstack(_lat)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(lon, lat, "-");